In [1]:
import numpy as np
import pandas as pd
import optuna
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.manifold import TSNE
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
#from geomloss import SamplesLoss
from torch.autograd import Function
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.nn.functional import normalize
#from torchmetrics.classification import BinaryAccuracy
#from torchmetrics.classification import BinaryF1Score
torch.manual_seed(0)

In [2]:
class TarNet(nn.Module):
    def __init__(self,params):
        super(TarNet, self).__init__()
        self.encoder1 = nn.Linear(25, params['RL11'])
        self.encoder2 = nn.Linear(params['RL11'], params['RL21'])
        self.encoder3 = nn.Linear(params['RL21'], params['RL32'])

        self.regressor1_y0 = nn.Sequential(
            nn.Linear(params['RL32'], params['RG012']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressor2_y0 = nn.Sequential(
            nn.Linear(params['RG012'], params['RG022']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressorO_y0 = nn.Linear(params['RG022'], 1)

        self.regressor1_y1 = nn.Sequential(
            nn.Linear(params['RL32'], params['RG112']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressor2_y1 = nn.Sequential(
            nn.Linear(params['RG112'], params['RG122']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressorO_y1 = nn.Linear(params['RG122'], 1)


    def forward(self, inputs):
        x = nn.functional.elu(self.encoder1(inputs))
        x = nn.functional.elu(self.encoder2(x))
        phi = nn.functional.elu(self.encoder3(x))

        out_y0 = self.regressor1_y0(phi)
        out_y0 = self.regressor2_y0(out_y0)
        y0 = self.regressorO_y0(out_y0)

        out_y1 = self.regressor1_y1(phi)
        out_y1 = self.regressor2_y1(out_y1)
        y1 = self.regressorO_y1(out_y1)

        concat = torch.cat((y0, y1), 1)
        return concat

In [3]:
def objective(trial,i):

    params = {
          'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
          'optimizer': trial.suggest_categorical("optimizer", ["Adam", "SGD"]),
          'batch_size':trial.suggest_int('batch_size', 8, 256),
          'RL11':trial.suggest_int('RL11', 16, 512),
          'RL21': trial.suggest_int('RL21', 16, 512),
          'RL32': trial.suggest_int('RL32', 16, 512),
          'RG012':trial.suggest_int('RG012', 16, 512),
        'RG022':trial.suggest_int('RG022', 16, 512),
        'RG112':trial.suggest_int('RG112', 16, 512),
        'RG122':trial.suggest_int('RG122', 16, 512),
          
          }

    model = TarNet(params)

    pehe,model= train_evaluate(params, model, trial,i)

    return pehe

In [4]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

In [5]:
def get_data(data_type,file_num):

    if(data_type=='train'):
        data=pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_train_{file_num}.csv")
    else:
        data = pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_test_{file_num}.csv")

    x_data=pd.concat([data.iloc[:,0], data.iloc[:, 1:30]], axis = 1)
    #x_data=data.iloc[:, 5:30]
    y_data=data.iloc[:, 1]
    return x_data,y_data

In [6]:
def get_dataloader(x_data,y_data,batch_size):

    x_train_sr=x_data[x_data['treatment']==0]
    y_train_sr=y_data[x_data['treatment']==0]
    x_train_tr=x_data[x_data['treatment']==1]
    y_train_tr=y_data[x_data['treatment']==1]


    train_data_sr = Data(np.array(x_train_sr), np.array(y_train_sr))
    train_dataloader_sr = DataLoader(dataset=train_data_sr, batch_size=batch_size)

    train_data_tr = Data(np.array(x_train_tr), np.array(y_train_tr))
    train_dataloader_tr = DataLoader(dataset=train_data_tr, batch_size=batch_size)


    return train_dataloader_sr, train_dataloader_tr

In [7]:
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = torch.sum((1. - t_true) * torch.square(y_true - y0_pred))
    loss1 = torch.sum(t_true * torch.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses instead of tf.reduce_mean.
    #They should be equivalent but it's possible that having larger gradients accelerates convergence.
    #You can always try changing it!
    return loss0 + loss1

In [8]:
def cal_pehe(data,y,model):
    #data,y=get_data('test',i)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    data=data.to_numpy()
    data=torch.from_numpy(data.astype(np.float32)).to(device)



    concat_pred=model(data[:,5:30])
    #dont forget to rescale the outcome before estimation!
    #y0_pred = data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
    #y1_pred = data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
    cate_pred=concat_pred[:,1]-concat_pred[:,0]
    cate_true=data[:,4]-data[:,3] #Hill's noiseless true values


    cate_err=torch.mean( torch.square( ( (cate_true) - (cate_pred) ) ) )

    return torch.sqrt(cate_err).item()


In [9]:


def loss_cal(X_data,y_data,net,device):
    
    x_train_sr=X_data[X_data['treatment']==0]
    y_train_sr=y_data[X_data['treatment']==0]
    x_train_tr=X_data[X_data['treatment']==1]
    y_train_tr=y_data[X_data['treatment']==1]
    xs_t=x_train_sr.iloc[:,0].to_numpy()
    xt_t=x_train_tr.iloc[:,0].to_numpy()
    
    xs=x_train_sr.iloc[:,5:30].to_numpy()
    xt=x_train_tr.iloc[:,5:30].to_numpy()
    xs_t=torch.from_numpy(xs_t.astype(np.float32))
    xt_t=torch.from_numpy(xt_t.astype(np.float32))
    y_train_sr=y_train_sr.to_numpy()
    y_train_tr=y_train_tr.to_numpy()
    xs=torch.from_numpy(xs.astype(np.float32))
    xt=torch.from_numpy(xt.astype(np.float32))
    
    y_train_sr=torch.from_numpy(y_train_sr.astype(np.float32))
    y_train_tr=torch.from_numpy(y_train_tr.astype(np.float32))
    
    
    input_data=torch.cat((xs,xt),0).to(device)
    true_y=torch.unsqueeze(torch.cat((y_train_sr,y_train_tr),0), dim=1).to(device)
    true_t=torch.unsqueeze(torch.cat((xs_t,xt_t),0), dim=1).to(device)
    
    
    concat_true=torch.cat((true_y,true_t),1)
    concat_pred=net(input_data)
    loss=regression_loss(concat_true, concat_pred)
    loss_2=y_MSE(concat_pred[0],concat_pred[1])
    return loss.item()

def cf_loss(xs,xt):

        #col =  ["treatment", "y_factual", "y_cfactual",]
        #for i in range(1,28):
        #    col.append("x"+str(i))

        #df_datac=pd.DataFrame(xs.numpy(),columns=col)
        #df_datat=pd.DataFrame(xt.numpy(),columns=col)
        
                
        PhiC=xs[:,5:30]
        PhiT=xt[:,5:30]
              
        
        dists = torch.sqrt(torch.cdist(PhiC, PhiT))
        #c_index=torch.argmin(dists, dim=0).tolist()
        #t_index=torch.argmin(dists, dim=1).tolist()
        #yT_nn=df_datac.iloc[c_index]['y_factual']
        #yC_nn=df_datat.iloc[t_index]['y_factual']
        
        
        c_index=torch.argmin(dists, dim=0)
        t_index=torch.argmin(dists, dim=1)
        yT_nn=xs[c_index,1]
        yC_nn=xt[t_index,1]
        
        
        #yT_nn=yT_nn.to_numpy()
        #yT_nn=torch.from_numpy(yT_nn.astype(np.float32))
        #yC_nn=yC_nn.to_numpy()
        #yC_nn=torch.from_numpy(yC_nn.astype(np.float32))
       
        return yC_nn,yT_nn
    
def cf_loss_prop(xs,xt):

        #col =  ["treatment", "y_factual", "y_cfactual",]
        #for i in range(1,28):
        #    col.append("x"+str(i))

        #df_datac=pd.DataFrame(xs.numpy(),columns=col)
        #df_datat=pd.DataFrame(xt.numpy(),columns=col)
        
                
        PhiC=xs[:,5:30]
        PhiT=xt[:,5:30]
              
        
        dists = torch.sqrt(torch.cdist(PhiC, PhiT))
        #c_index=torch.argmin(dists, dim=0).tolist()
        #t_index=torch.argmin(dists, dim=1).tolist()
        #yT_nn=df_datac.iloc[c_index]['y_factual']
        #yC_nn=df_datat.iloc[t_index]['y_factual']
        
        
        c_index=torch.argmin(dists, dim=0)
        t_index=torch.argmin(dists, dim=1)
        yT_nn=xs[c_index,1]
        yC_nn=xt[t_index,1]
        
        
        #yT_nn=yT_nn.to_numpy()
        #yT_nn=torch.from_numpy(yT_nn.astype(np.float32))
        #yC_nn=yC_nn.to_numpy()
        #yC_nn=torch.from_numpy(yC_nn.astype(np.float32))
       
        return yC_nn,yT_nn

In [10]:

y_MSE=nn.MSELoss()
#criterion_reg=regression_loss(concat_true,concat_pred)
epochs=300
#batch_size=32

In [12]:
train_loss=[]
val_loss=[]
pehe_error=[]
num_files=2
lambda_=0.1
def train_evaluate(param, model, trial,file_num):
    #for nf in range(1,num_files):
    x_data,y_data=get_data('train',file_num)
    X_train, X_val,y_train, y_val = train_test_split(x_data,y_data ,
                                       random_state=42, 
                                       test_size=0.20)
    
    #net=TarNet(25,.01)
    #opt_net = torch.optim.Adam(net.parameters(), lr=1e-4)
    
   
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])
    
    #if use_cuda:

        #model = model.cuda()
    model = model.to(device)
        #criterion = criterion.cuda()

    for ep in range(1,epochs+1 ):

        train_dataloader_sr, train_dataloader_tr=get_dataloader(X_train,y_train,param['batch_size'])

        for batch_idx, (train_source_data, train_target_data) in enumerate(zip(train_dataloader_sr, train_dataloader_tr)):

            xs,ys=train_source_data
            xt,yt=train_target_data
            yC_nn,yT_nn=cf_loss(xs,xt)
            #print(xs)

            xs_train=xs[:,5:30]
            xt_train=xt[:,5:30]

            train_x=torch.cat((xs_train,xt_train),0).to(device)
            train_y=torch.unsqueeze(torch.cat((ys,yt),0), dim=1).to(device)
            true_t=torch.unsqueeze(torch.cat((xs[:,0],xt[:,0]),0), dim=1).to(device)
            concat_true=torch.cat((train_y,true_t),1).to(device)
            concat_pred=model(train_x).to(device)

            model.zero_grad()

            #source_mse=criterion_reg(y0,ys)
            #target_mse=criterion_reg(y1,yt)
            #cf_los=(y_MSE(concat_pred[0:xs_train.shape[0],1],yC_nn.to(device)))+(y_MSE(concat_pred[xs_train.shape[0]:,0],yT_nn.to(device)))
            cf_los_1=torch.sum(torch.abs(concat_pred[0:xs_train.shape[0],1]-yC_nn.to(device)))
            cf_los_2=torch.sum(torch.abs(concat_pred[xs_train.shape[0]:,0]-yT_nn.to(device)))
            cf_los=cf_los_1+cf_los_2
            #combined loss
            combined_loss=(lambda_*regression_loss(concat_true,concat_pred))+(1-lambda_)*cf_los #add tradeoff term here with between two
            #losses
                        #print('Training loss: ',combined_loss.item())
            # backward propagation
            combined_loss.backward()

            # optimize
            optimizer.step()
        #train_loss.append(loss_cal(X_train,y_train,net))
        #val_loss.append(loss_cal(X_val,y_val,net))
        
        # Add prune mechanism
        #trial.report(accuracy, ep)

        #if trial.should_prune():
        #   raise optuna.exceptions.TrialPruned()
            
    #return cal_pehe(X_val,y_val,model),model
    return loss_cal(X_val,y_val,model,device),model

        
        

In [13]:
pehe_total=[]
for i in range(1,101):
    func = lambda trial: objective(trial, i)
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(func, n_trials=50)
    best_trial = study.best_trial
    best_model=TarNet(study.best_trial.params)
    best_val,model=train_evaluate(study.best_trial.params, best_model, study.best_trial,i)
    data,y=get_data('test',i)
    pehe=cal_pehe(data,y,model)

    pehe_total.append(pehe)


[I 2023-05-18 12:02:07,396] A new study created in memory with name: no-name-1dab97b5-f270-4690-9afa-c482fd841415
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[I 2023-05-18 12:02:12,667] Trial 0 finished with value: 435.0091552734375 and parameters: {'learning_rate': 5.6115164153345e-05, 'optimizer': 'Adam', 'batch_size': 157, 'RL11': 93, 'RL21': 93, 'RL32': 44, 'RG012': 446, 'RG022': 314, 'RG112': 367, 'RG122': 26}. Best is trial 0 with value: 435.0091552734375.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Tri

[I 2023-05-18 12:02:43,487] Trial 12 finished with value: 233.5113983154297 and parameters: {'learning_rate': 0.00014595432633183366, 'optimizer': 'SGD', 'batch_size': 118, 'RL11': 191, 'RL21': 505, 'RL32': 221, 'RG012': 119, 'RG022': 511, 'RG112': 142, 'RG122': 163}. Best is trial 11 with value: 199.67376708984375.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 12:02:45,430] Trial 13 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[I 2023-05-18 12:03:19,463] Trial 30 finished with value: 249.4140167236328 and parameters: {'learning_rate': 5.8560747973134955e-05, 'optimizer': 'SGD', 'batch_size': 251, 'RL11': 227, 'RL21': 250, 'RL32': 447, 'RG012': 80, 'RG022': 450, 'RG112': 56, 'RG122': 239}. Best is trial 11 with value: 199.67376708984375.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[I 2023-05-18 12:03:21,313] Trial 31 fin

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[I 2023-05-18 12:03:58,309] Trial 43 finished with value: 260.1180419921875 and parameters: {'learning_rate': 0.0001359538600069858, 'optimizer': 'SGD', 'batch_size': 131, 'RL11': 80, 'RL21': 447, 'RL32': 259, 'RG012': 382, 'RG022': 148, 'RG112': 55, 'RG122': 374}. Best is trial 11 with value: 199.67376708984375.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 12:04:00,183] Trial 44 fail

[I 2023-05-18 12:04:31,461] Trial 6 finished with value: 259.3167724609375 and parameters: {'learning_rate': 1.913588048769229e-05, 'optimizer': 'Adam', 'batch_size': 253, 'RL11': 399, 'RL21': 114, 'RL32': 18, 'RG012': 421, 'RG022': 367, 'RG112': 378, 'RG122': 399}. Best is trial 2 with value: 188.65341186523438.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[I 2023-05-18 12:04:33,658] Trial 7 finished with value: 427.24566650390625 and parameters: {'learning_rate': 1.4063366777718176e-05, 'optimizer': 'Adam', 'batch_size': 222, 'RL11': 325, 'RL21': 180, 'RL32': 47, 'RG012': 170, 'RG022': 177, 'RG112': 378, 'RG122': 332}. Best is trial 2 with value: 188.65341186523438.
/usr/local/lib/python3.6/dist-packages/ipykern

...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 15:17:42,405] Trial 28 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 15:17:44,558] Trial 29 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feat

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 15:18:18,449] Trial 46 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 15:18:20,363] Trial 47 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feat

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[I 2023-05-18 15:18:55,044] Trial 10 finished with value: 211.64414978027344 and parameters: {'learning_rate': 0.0002174629603640568, 'optimizer': 'Adam', 'batch_size': 181, 'RL11': 484, 'RL21': 339, 'RL32': 448, 'RG012': 317, 'RG022': 36, 'RG112': 236, 'RG122': 181}. Best is trial 8 with value: 198.14830017089844.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[I 2023-05-18 15:18:58,120] Trial 11 fi

[W 2023-05-18 15:19:26,399] Trial 26 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 15:19:28,204] Trial 27 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 15:19:30,042] Trial 28 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist

[W 2023-05-18 15:20:03,775] Trial 43 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 15:20:05,827] Trial 44 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
[W 2023-05-18 15:20:07,879] Trial 45 failed because of the following error: The value nan is not acceptable.
/usr/local/lib/python3.6/dist

In [14]:
print(np.mean(pehe_total))

nan


In [16]:
#np.mean(pehe_total[0:99])

In [18]:
np.savetxt("V3_CFR_y_loss_1_100_(IHDPa-Hyper_val_300ep_outsample).csv", pehe_total,delimiter =", ", fmt ='% s')

In [44]:
#for key, value in best_trial.params.items():
#    print("{}: {}".format(key, value))

[]

In [ ]:
#ate_pred=torch.mean(cate_pred)
#print("Estimated ATE (True is 4):", ate_pred.detach().numpy(),'\n\n')

#print("Individualized CATE Estimates: BLUE")
#print(pd.Series(cate_pred.detach().numpy()).plot.kde(color='blue'))
#print("Individualized CATE True: Green")
#print(pd.Series(cate_true.detach().numpy()).plot.kde(color='green'))

#print("\nError CATE Estimates: RED")
#print(pd.Series(cate_pred.detach().numpy()-cate_true.detach().numpy()).plot.kde(color='red'))